# Favorita Store Sales - Exploratory Data Analysis

This notebook provides interactive exploration of the Corporación Favorita grocery sales dataset.

In [ ]:
# Add project root to path
import sys
sys.path.insert(0, '..')

# Standard imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

warnings.filterwarnings('ignore')
plt.style.use('seaborn-v0_8-whitegrid')

# Project imports
from src.data_prep import load_master_dataframe, create_master_dataframe
from src.eda import (
    get_basic_stats,
    analyze_sales_distribution,
    analyze_seasonality,
    analyze_promotion_effect,
    analyze_oil_correlation,
    analyze_holiday_effect,
    analyze_store_patterns,
    perform_stl_decomposition
)

print('Imports complete!')

## 1. Load Data

In [ ]:
# Load master DataFrame (create if doesn't exist)
try:
    df = load_master_dataframe()
except FileNotFoundError:
    print('Master DataFrame not found. Creating from raw data...')
    df = create_master_dataframe()

print(f'Dataset shape: {df.shape}')
df.head()

## 2. Basic Statistics

In [ ]:
stats = get_basic_stats(df)

print('DATASET OVERVIEW')
print('='*50)
print(f"Time Range: {stats['time_range'][0]} to {stats['time_range'][1]}")
print(f"Number of Days: {stats['n_days']:,}")
print(f"Number of Stores: {stats['n_stores']}")
print(f"Number of Families: {stats['n_families']}")
print(f"Total Records: {stats['total_rows']:,}")
print(f"Zero Sales Ratio: {stats['zero_sales_ratio']*100:.1f}%")
print(f"Memory Usage: {stats['memory_mb']:.1f} MB")

In [ ]:
# Data types and missing values
print('\nColumn Info:')
df.info()

In [ ]:
# Summary statistics
df.describe()

## 3. Sales Distribution

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Regular scale
df[df['sales'] > 0]['sales'].hist(bins=100, ax=axes[0], alpha=0.7, edgecolor='black')
axes[0].set_xlabel('Sales')
axes[0].set_ylabel('Frequency')
axes[0].set_title('Sales Distribution (Non-Zero)')

# Log scale
df[df['sales'] > 0]['sales'].apply(np.log1p).hist(bins=100, ax=axes[1], alpha=0.7, edgecolor='black')
axes[1].set_xlabel('Log(Sales + 1)')
axes[1].set_ylabel('Frequency')
axes[1].set_title('Sales Distribution (Log Scale)')

plt.tight_layout()
plt.show()

In [ ]:
# Sales by family
family_sales = df.groupby('family')['sales'].agg(['mean', 'median', 'sum']).sort_values('sum', ascending=False)
print('Sales by Product Family (sorted by total):')
family_sales

## 4. Time Series Visualization

In [ ]:
# Total daily sales
daily_sales = df.groupby('date')['sales'].sum()

fig, ax = plt.subplots(figsize=(16, 6))
ax.plot(daily_sales.index, daily_sales.values, linewidth=0.5, alpha=0.8, label='Daily Sales')
ax.plot(daily_sales.rolling(7).mean(), linewidth=2, color='red', label='7-day MA')
ax.set_xlabel('Date')
ax.set_ylabel('Total Sales')
ax.set_title('Total Daily Sales Over Time')
ax.legend()
plt.show()

## 5. Seasonality Patterns

In [ ]:
# Day of week pattern
dow_sales = df.groupby('dayofweek')['sales'].mean()
dow_names = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun']

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

axes[0].bar(dow_names, dow_sales.values, color='steelblue', edgecolor='black')
axes[0].set_xlabel('Day of Week')
axes[0].set_ylabel('Average Sales')
axes[0].set_title('Average Sales by Day of Week')

# Monthly pattern
monthly_sales = df.groupby('month')['sales'].mean()
axes[1].plot(monthly_sales.index, monthly_sales.values, 'o-', linewidth=2, markersize=8)
axes[1].set_xlabel('Month')
axes[1].set_ylabel('Average Sales')
axes[1].set_title('Average Sales by Month')
axes[1].set_xticks(range(1, 13))

plt.tight_layout()
plt.show()

In [ ]:
# Heatmap: Day of Week x Month
dow_month = df.groupby(['dayofweek', 'month'])['sales'].mean().unstack()

fig, ax = plt.subplots(figsize=(12, 6))
sns.heatmap(dow_month, annot=True, fmt='.0f', cmap='YlOrRd', ax=ax)
ax.set_xlabel('Month')
ax.set_ylabel('Day of Week')
ax.set_yticklabels(dow_names)
ax.set_title('Average Sales: Day of Week × Month')
plt.show()

## 6. Promotion Analysis

In [ ]:
# Promotion effect by family
promo_effect = df.groupby(['family', 'onpromotion'])['sales'].mean().unstack()
promo_effect.columns = ['No Promo', 'Promo']
promo_effect['Lift %'] = (promo_effect['Promo'] / promo_effect['No Promo'] - 1) * 100
promo_effect = promo_effect.sort_values('Lift %', ascending=False)

print('Promotion Effect by Family:')
promo_effect.round(2)

In [ ]:
# Visualize promotion lift
fig, ax = plt.subplots(figsize=(12, 8))
colors = ['green' if x > 0 else 'red' for x in promo_effect['Lift %']]
ax.barh(range(len(promo_effect)), promo_effect['Lift %'], color=colors)
ax.set_yticks(range(len(promo_effect)))
ax.set_yticklabels(promo_effect.index)
ax.set_xlabel('Promotion Lift (%)')
ax.set_title('Promotion Effect by Product Family')
ax.axvline(x=0, color='black', linestyle='-', linewidth=0.5)
plt.tight_layout()
plt.show()

## 7. Oil Price Analysis

In [ ]:
if 'dcoilwtico' in df.columns:
    # Daily aggregates
    daily = df.groupby('date').agg({'sales': 'sum', 'dcoilwtico': 'first'})
    
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # Scatter plot
    axes[0].scatter(daily['dcoilwtico'], daily['sales'], alpha=0.3, s=10)
    axes[0].set_xlabel('Oil Price (WTI)')
    axes[0].set_ylabel('Total Daily Sales')
    corr = daily['sales'].corr(daily['dcoilwtico'])
    axes[0].set_title(f'Oil Price vs Total Sales (r={corr:.3f})')
    
    # Time series
    ax1 = axes[1]
    ax2 = ax1.twinx()
    ax1.plot(daily.index, daily['sales'], 'b-', alpha=0.5, linewidth=0.5, label='Sales')
    ax2.plot(daily.index, daily['dcoilwtico'], 'r-', alpha=0.5, linewidth=0.5, label='Oil')
    ax1.set_ylabel('Total Sales', color='blue')
    ax2.set_ylabel('Oil Price', color='red')
    ax1.set_title('Sales and Oil Price Over Time')
    
    plt.tight_layout()
    plt.show()
else:
    print('Oil price column not found')

## 8. Store Analysis

In [ ]:
# Sales by store type
if 'type' in df.columns:
    store_type_sales = df.groupby('type')['sales'].agg(['mean', 'sum']).sort_values('sum', ascending=False)
    print('Sales by Store Type:')
    print(store_type_sales.round(2))
    print()

# Sales by city
if 'city' in df.columns:
    city_sales = df.groupby('city')['sales'].sum().sort_values(ascending=False).head(10)
    print('Top 10 Cities by Total Sales:')
    print(city_sales.round(2))

In [ ]:
# Sales by cluster
if 'cluster' in df.columns:
    cluster_sales = df.groupby('cluster')['sales'].mean().sort_values(ascending=False)
    
    fig, ax = plt.subplots(figsize=(12, 5))
    ax.bar(cluster_sales.index.astype(str), cluster_sales.values, color='steelblue', edgecolor='black')
    ax.set_xlabel('Cluster')
    ax.set_ylabel('Average Sales')
    ax.set_title('Average Sales by Store Cluster')
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

## 9. Zero-Inflation Analysis

In [ ]:
# Zero sales ratio by family
zero_ratio = df.groupby('family').apply(lambda x: (x['sales'] == 0).mean()).sort_values(ascending=False)

fig, ax = plt.subplots(figsize=(12, 6))
ax.barh(range(len(zero_ratio)), zero_ratio.values * 100, color='coral', edgecolor='black')
ax.set_yticks(range(len(zero_ratio)))
ax.set_yticklabels(zero_ratio.index)
ax.set_xlabel('Zero Sales Ratio (%)')
ax.set_title('Zero-Inflation by Product Family')
plt.tight_layout()
plt.show()

## 10. Interactive Exploration

Use the cells below for your own exploration!

In [ ]:
# Your exploration here
# Example: Filter to specific store/family
# store_1_grocery = df[(df['store_nbr'] == 1) & (df['family'] == 'GROCERY I')]
# store_1_grocery.head()